##### Copyright 2019 The TensorFlow Authors.

In [ ]:
# Apache License
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 Quickstart and Learn By Numbers

### Modified by Katherine Haynes and Richi Rodriguez

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

The introduction is followed by:
1. Routine to plot all mis-matches.
2. Sensitivity study of various epochs.
3. Sensitivity study of various network widths and depths.
4. Results using Sobel edge detection.

## Load Libraries

In [22]:
# Load Libraries
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import cv2
from IPython.core.display import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas
from random import randint
import time

## Load Data

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [2]:
# Load MNIST
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Print image shape and number of training/testing samples.

In [3]:
# Print MNIST Info
ssize=x_train.shape
tsize=x_test.shape
nTrain=ssize[0]
nImage=ssize[1]
nTest=tsize[0]
print('Image Size: {} x {}, Training Samples: {}, Testing Samples: {}'.format(
    ssize[1],ssize[2],nTrain,nTest))

Image Size: 28 x 28, Training Samples: 60000, Testing Samples: 10000


Plot sample image.

In [4]:
# Plot Sample Image
plotSample = False
if plotSample:
   nRef = randint(0,nTrain-1)
   myImage = cv2.resize(x_train[nRef,:,:],dsize=(nImage*10,nImage*10)) 
   string1 = 'Truth: ' + str(y_train[nRef])

   winName = 'Sample MNIST Image'
   font = cv2.FONT_HERSHEY_SIMPLEX
   bottomLeftCornerOfText = (10,20)
   fontScale = 0.5
   fontColor = (255,255,255)
   lineType = 2
   cv2.putText(myImage,string1,
            bottomLeftCornerOfText, font, fontScale, fontColor, lineType)

   cv2.imshow(winName,myImage)
   print("Press any key on image to continue...")
   cv2.waitKey(0)
   cv2.destroyAllWindows()
   cv2.waitKey(1)
   print("You will have to reselect notebook to continue...")

## Define Plotting Routines

In [5]:
# Plotting Routine To Display Mis-Matches
def display_mismatch(X, predict, truth, waitK=400):
    badref = np.where(predict != truth)[0]
    nMismatch = badref.shape[0]
    nImage1 = X.shape[1]
    nImage2 = X.shape[2]

    print('Displaying {} mismatches'.format(nMismatch))
    print('Press q to exit at any time...')
    for ref in badref:
        img = cv2.resize(X[ref,:,:],dsize=(nImage1*10,nImage2*10))
        stringT = 'Truth: ' + str(truth[ref])
        stringP = 'Predicted: ' + str(predict[ref])
        
        winName = 'Mismatched MNIST Image'
        font = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerT = (10,20)
        bottomLeftCornerP = (10,270)
        fontScale = 0.5
        fontColorT = (200,200,200)
        fontColorP = (100,100,255)
        lineType = 2
        cv2.putText(img,stringT,
            bottomLeftCornerT, font, fontScale, fontColorT, lineType)
        cv2.putText(img,stringP,
            bottomLeftCornerP, font, fontScale, fontColorP, lineType)
        cv2.imshow(winName,img)
        key = cv2.waitKey(waitK)
        if key == ord('q'):
            break

    cv2.destroyAllWindows()
    cv2.waitKey(1)
    print('Done')

# --Tutorial Section--

In [6]:
runTutorial = False

## Build Model

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28),dtype='float64'),
  tf.keras.layers.Dense(128, activation='relu',dtype='float64'),
  tf.keras.layers.Dropout(0.2,dtype='float64'),
  tf.keras.layers.Dense(10,dtype='float64')
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [8]:
if runTutorial:
   predictions = model(x_train[:1]).numpy()
   print(predictions)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [9]:
if runTutorial:
   tf.nn.softmax(predictions).numpy()

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [11]:
if runTutorial:
   loss_fn(y_train[:1], predictions).numpy()

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train Model

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [13]:
if runTutorial:
   nEpochs=1
   model.fit(x_train, y_train, epochs=nEpochs)

## Evaluate Model

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".

In [14]:
if runTutorial:
   loss, accuracy = model.evaluate(x_test,  y_test, verbose=2)
   print("The image classifier is now trained to {:.3f}% accuracy on the test dataset.".format(accuracy*100))

## Use Model

To return a probability, wrap the trained model and attach the softmax to it:

In [15]:
if runTutorial:
   probability_model = tf.keras.Sequential([
     model,
     tf.keras.layers.Softmax(dtype='float64')
   ])

In [16]:
if runTutorial:
   classes = np.array([0,1,2,3,4,5,6,7,8,9])
   y_predict = probability_model(x_test).numpy()
   y_classes = classes[y_predict.argmax(axis=1)] #.reshape((-1, 1))

## Display Mis-Matches

In [17]:
if runTutorial:
   display_mismatch(x_test, y_classes, y_test)

# --Epoch Sensitivity Study--

In [18]:
nEpochs = [1,2,4,6,8,10,20,40]
studyEpochs = False
saveEpochs = True

### Train and Test Model

In [30]:
# Train and Test Model Varying Epochs
if studyEpochs:
   results=[]
   for i in nEpochs:
       start_time = time.time()
       model.fit(x_train, y_train, epochs=i)
       loss, accuracy = model.evaluate(x_test,  y_test, verbose=2)
       stop_time = time.time()
       results.append([i,accuracy*100.,loss,stop_time-start_time])
       print()
        
   eResults = pandas.DataFrame(results, 
              columns=('# Epochs', 'Accuracy', 'Loss', 'Time'))
   myX = eResults['# Epochs']
   myA = eResults['Accuracy']
   myT = eResults['Time']

In [41]:
# Show previously calculated results
Image(filename="epochsT.png", width=400, height=600)

### Display Results Table

In [52]:
# Display Epoch Results Table
if studyEpochs:
    display((eResults.sort_values(by=['# Epochs'],ascending=True)))
else:
    myImage = cv2.imread("epochsT.png")
    cv2.imshow('Epochs',myImage)
    cv2.waitKey(0)
    print("Press any key on image to continue...")
    cv2.destroyWindow('Epochs')
    cv2.waitKey(1)
    print("You will have to reselect notebook to continue...")

Press any key on image to continue...
You will have to reselect notebook to continue...


### Display Results Plot

In [54]:
# Display Epoch Results Plot
if studyEpochs:
    fig, axs = plt.subplots(nrows=1,ncols=2)
    fig.set_size_inches(7,3.5)
    
    axs[0].plot(myX, myA)
    axs[0].set_title('Accuracy')
    axs[0].set_xlabel('# Epochs')
    axs[0].set_ylabel('%')
    
    axs[1].plot(myX, myT)
    axs[1].set_title('Time (s)')
    axs[1].set_xlabel('# Epochs')
    #axs[1].set_ylabel('s')
    #fig.tight_layout()
    
    if saveEpochs:
        plt.savefig('epochs.png')
else:
    myImage = cv2.imread("epochsF.png")
    cv2.imshow('Epochs',myImage)
    cv2.waitKey(0)
    print("Press any key on image to continue...")
    cv2.destroyWindow('Epochs')
    cv2.waitKey(1)
    print("You will have to reselect notebook to continue...")

Press any key on image to continue...
You will have to reselect notebook to continue...


# --Model Width and Depth Study--

In [55]:
studyModels = True
saveModels = True

In [63]:
modelSetups = {
    1: tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28),dtype='float64'),
    tf.keras.layers.Dense(128, activation='relu',dtype='float64'),
    tf.keras.layers.Dropout(0.2,dtype='float64'),
    tf.keras.layers.Dense(10,dtype='float64')])}

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [77]:
if studyModels:
   results = []

   keyList = list(modelSetups.keys())
   for i in keyList:
       start_time = time.time()
       model = modelSetups.get(i)
       model.compile(optimizer='adam',
                     loss=loss_fn,
                     metrics=['accuracy'])
       model.fit(x_train, y_train, epochs=1)
       loss, accuracy = model.evaluate(x_test,  y_test, verbose=2)
       stop_time = time.time()
    
       results.append([i,accuracy*100.,loss,stop_time-start_time])
       print()
        
   mResults = pandas.DataFrame(results, 
              columns=('Ref #', 'Accuracy', 'Loss', 'Time'))
   myMX = mResults['Ref #']
   myMA = mResults['Accuracy']
   myMT = mResults['Time']

Train on 60000 samples
60000/60000 [==============================] - 7s 123us/sample - loss: 0.0761 - accuracy: 0.9769
10000/1 - 1s - loss: 0.0432 - accuracy: 0.9748



In [79]:
# Display Epoch Results Table
if studyModels:
    display((mResults.sort_values(by=['Ref #'],ascending=True)))
#else:
    #myImage = cv2.imread("modelsT.png")
    #cv2.imshow('Models',myImage)
    #cv2.waitKey(0)
    #print("Press any key on image to continue...")
    #cv2.destroyWindow('Models')
    #cv2.waitKey(1)
    #print("You will have to reselect notebook to continue...")

,Ref #,Accuracy,Loss,Time
0,1,97.479999,0.084587,8.035983
